In [ ]:
##### REQUIRES THE DATAFRAME FOLDER TO BE NAMED 'Cohorts', WHICH INCLUDES ALL PRECOMPUTED DATAFRAMES #####
import fiber
from fiber.cohort import Cohort
from fiber.condition import Patient, MRNs
from fiber.condition import Diagnosis
from fiber.condition import Measurement, Encounter, Drug
from fiber.storage import yaml as fiberyaml
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
import os
from functools import reduce
from sklearn.linear_model import LinearRegression

In [ ]:
Cases = pq.read_table('Cohorts/Case/Case_BMI_BP_Pulse_Labs_not_imputed.parquet').to_pandas()
Cases_meta = pq.read_table('Cohorts/Case/Case_First_Model.parquet').to_pandas()


In [ ]:
Cases_meta = Cases_meta[["gender", "year_of_birth"]]

In [ ]:
Case_complete = Cases_meta.merge(Cases, right_index = True, left_index=True, how="inner")

In [236]:
Case_complete.columns
#column_name = "CARBON_DIOXIDE_BLD_max"

Index(['gender', 'year_of_birth', 'BMI_min', 'BMI_max', 'BMI_mean', 'SBP_min',
       'SBP_max', 'SBP_mean', 'pulse_rate_min', 'pulse_rate_max',
       'pulse_rate_mean', 'DBP_min', 'DBP_max', 'DBP_mean',
       'CARBON_DIOXIDE_BLD_min', 'CARBON_DIOXIDE_BLD_max',
       'CARBON_DIOXIDE_BLD_mean', 'UREA_NITROGEN-BLD_min',
       'UREA_NITROGEN-BLD_max', 'UREA_NITROGEN-BLD_mean',
       'CREATININE-SERUM_min', 'CREATININE-SERUM_max', 'CREATININE-SERUM_mean',
       'HEMATOCRIT_min', 'HEMATOCRIT_max', 'HEMATOCRIT_mean', 'GLUCOSE_min',
       'GLUCOSE_max', 'GLUCOSE_mean'],
      dtype='object')

In [217]:
Case_impute_CARBON = Case_complete[['gender', 'year_of_birth', 'BMI_min', 'BMI_max', 'BMI_mean',column_name ]]

In [218]:
Case_impute_CARBON = pd.get_dummies(Case_impute_CARBON, columns=["gender"])

In [219]:
Case_impute_CARBON= Case_impute_CARBON.drop(["gender_Unknown"], axis=1)

In [220]:
data_without_null = Case_impute_CARBON.dropna()

In [221]:
data_with_null = Case_impute_CARBON.drop(data_without_null.index)

In [222]:
train_data_x = data_without_null[['year_of_birth', 'BMI_min', 'BMI_max', 'BMI_mean',
       'gender_Female', 'gender_Male']]

In [223]:
train_data_y = data_without_null[[column_name]]

In [224]:
linreg = LinearRegression()

In [225]:
linreg.fit(train_data_x, train_data_y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [226]:
test_data = data_with_null[['year_of_birth', 'BMI_min', 'BMI_max', 'BMI_mean', 'gender_Female', 'gender_Male']]

In [227]:
x  = pd.DataFrame(linreg.predict(test_data))[0]


In [228]:
data_with_null =data_with_null.reset_index()

In [229]:
data_with_null = data_with_null.merge(x, left_index=True, right_index=True, how="inner")

In [230]:
data_with_null = data_with_null.drop([column_name], axis=1)

In [231]:
data_with_null.rename(columns={0:column_name}, inplace=True
                     )

In [232]:
data_with_null =data_with_null.set_index("medical_record_number")

In [233]:
y = pd.concat([data_with_null, data_without_null])

/home/cremej01/anaconda3/envs/JonasPythonEnv/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [234]:
final = final.merge(y[column_name], left_index=True, right_index=True, how="inner")

In [254]:

Case_complete.columns
column_name = "GLUCOSE_mean"


# In[217]:


Case_impute_CARBON = Case_complete[['gender', 'year_of_birth', 'BMI_min', 'BMI_max', 'BMI_mean',column_name ]]


# In[218]:


Case_impute_CARBON = pd.get_dummies(Case_impute_CARBON, columns=["gender"])


# In[219]:


Case_impute_CARBON= Case_impute_CARBON.drop(["gender_Unknown"], axis=1)


# In[220]:


data_without_null = Case_impute_CARBON.dropna()


# In[221]:


data_with_null = Case_impute_CARBON.drop(data_without_null.index)


# In[222]:


train_data_x = data_without_null[['year_of_birth', 'BMI_min', 'BMI_max', 'BMI_mean',
       'gender_Female', 'gender_Male']]


# In[223]:


train_data_y = data_without_null[[column_name]]


# In[224]:


linreg = LinearRegression()


# In[225]:


linreg.fit(train_data_x, train_data_y)


# In[226]:


test_data = data_with_null[['year_of_birth', 'BMI_min', 'BMI_max', 'BMI_mean', 'gender_Female', 'gender_Male']]


# In[227]:


x  = pd.DataFrame(linreg.predict(test_data))[0]


# In[228]:


data_with_null =data_with_null.reset_index()


# In[229]:


data_with_null = data_with_null.merge(x, left_index=True, right_index=True, how="inner")


# In[230]:


data_with_null = data_with_null.drop([column_name], axis=1)


# In[231]:


data_with_null.rename(columns={0:column_name}, inplace=True)


# In[232]:


data_with_null =data_with_null.set_index("medical_record_number")


# In[233]:


y = pd.concat([data_with_null, data_without_null])


# In[234]:


final = final.merge(y[column_name], left_index=True, right_index=True, how="inner")


# In[235]:


final

/home/cremej01/anaconda3/envs/JonasPythonEnv/lib/python3.7/site-packages/ipykernel_launcher.py:106: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,BMI_min,BMI_max,BMI_mean,SBP_min,SBP_max,SBP_mean,pulse_rate_min,pulse_rate_max,pulse_rate_mean,DBP_min,...,UREA_NITROGEN-BLD_mean,CREATININE-SERUM_max,CREATININE-SERUM_mean,CREATININE-SERUM_min,HEMATOCRIT_min,HEMATOCRIT_max,HEMATOCRIT_mean,GLUCOSE_min,GLUCOSE_max,GLUCOSE_mean
medical_record_number,,,,,,,,,,,,,,,,,,,,,
1000012395,30.618890,34.739230,31.984547,126.0,152.0,139.000000,58.0,100.0,71.444444,76.0,...,13.266390,1.219338,1.063146,0.948871,38.580330,44.604893,41.489286,82.592455,148.871306,106.381766
1000227914,21.953105,22.670535,22.311820,110.0,150.0,126.666667,76.0,97.0,85.666667,70.0,...,12.133333,1.000000,0.780000,0.600000,35.800000,47.700000,43.682353,69.000000,110.000000,85.333333
1000508752,21.273691,23.504128,22.335818,100.0,151.0,127.631579,65.0,101.0,83.615385,54.0,...,15.994548,0.967333,0.852969,0.766009,34.750066,40.034262,37.334834,82.388110,126.834509,99.385362
1000519317,38.291528,38.291528,38.291528,120.0,120.0,120.000000,64.0,82.0,71.800000,84.0,...,17.513687,0.882304,0.859767,0.820258,38.063066,39.848864,38.993787,95.217777,111.735265,105.749216
1000560997,25.683551,27.983299,26.437846,133.0,144.0,138.500000,65.0,102.0,82.000000,91.0,...,14.098288,1.238332,1.077079,0.961860,38.446739,44.459870,41.330898,81.509447,146.050958,104.309957
1000604785,25.050945,27.700542,26.397279,110.0,132.0,120.000000,60.0,88.0,70.571429,60.0,...,23.000000,0.800000,0.800000,0.800000,40.200000,40.200000,40.200000,105.000000,105.000000,105.000000
1000614901,23.994118,25.885234,25.361511,156.0,156.0,156.000000,76.0,105.0,86.136364,85.0,...,20.263773,1.309287,1.178520,1.074972,40.459036,44.030210,42.177499,92.752113,133.406851,109.815598
1000628903,16.832422,29.223602,29.223602,97.0,140.0,115.285714,64.0,90.0,77.674556,65.0,...,51.000000,8.900000,8.900000,8.900000,43.000000,43.000000,43.000000,91.000000,91.000000,91.000000
100067499,23.674874,25.109734,24.272838,118.0,149.0,133.750000,62.0,108.0,85.105691,53.0,...,17.508111,1.279790,1.132793,1.023006,39.439830,44.198080,41.719881,87.175870,139.328809,106.935197


In [245]:
final1 = final.drop(["CREATININE-SERUM_min_x", "CREATININE-SERUM_min_y"], axis=1)

In [255]:
final.to_parquet("Cohorts/Case/Case_BMI_BP_Pulse_Labs_imputed.parquet")

In [256]:
final

,BMI_min,BMI_max,BMI_mean,SBP_min,SBP_max,SBP_mean,pulse_rate_min,pulse_rate_max,pulse_rate_mean,DBP_min,...,UREA_NITROGEN-BLD_mean,CREATININE-SERUM_max,CREATININE-SERUM_mean,CREATININE-SERUM_min,HEMATOCRIT_min,HEMATOCRIT_max,HEMATOCRIT_mean,GLUCOSE_min,GLUCOSE_max,GLUCOSE_mean
medical_record_number,,,,,,,,,,,,,,,,,,,,,
1000012395,30.618890,34.739230,31.984547,126.0,152.0,139.000000,58.0,100.0,71.444444,76.0,...,13.266390,1.219338,1.063146,0.948871,38.580330,44.604893,41.489286,82.592455,148.871306,106.381766
1000227914,21.953105,22.670535,22.311820,110.0,150.0,126.666667,76.0,97.0,85.666667,70.0,...,12.133333,1.000000,0.780000,0.600000,35.800000,47.700000,43.682353,69.000000,110.000000,85.333333
1000508752,21.273691,23.504128,22.335818,100.0,151.0,127.631579,65.0,101.0,83.615385,54.0,...,15.994548,0.967333,0.852969,0.766009,34.750066,40.034262,37.334834,82.388110,126.834509,99.385362
1000519317,38.291528,38.291528,38.291528,120.0,120.0,120.000000,64.0,82.0,71.800000,84.0,...,17.513687,0.882304,0.859767,0.820258,38.063066,39.848864,38.993787,95.217777,111.735265,105.749216
1000560997,25.683551,27.983299,26.437846,133.0,144.0,138.500000,65.0,102.0,82.000000,91.0,...,14.098288,1.238332,1.077079,0.961860,38.446739,44.459870,41.330898,81.509447,146.050958,104.309957
1000604785,25.050945,27.700542,26.397279,110.0,132.0,120.000000,60.0,88.0,70.571429,60.0,...,23.000000,0.800000,0.800000,0.800000,40.200000,40.200000,40.200000,105.000000,105.000000,105.000000
1000614901,23.994118,25.885234,25.361511,156.0,156.0,156.000000,76.0,105.0,86.136364,85.0,...,20.263773,1.309287,1.178520,1.074972,40.459036,44.030210,42.177499,92.752113,133.406851,109.815598
1000628903,16.832422,29.223602,29.223602,97.0,140.0,115.285714,64.0,90.0,77.674556,65.0,...,51.000000,8.900000,8.900000,8.900000,43.000000,43.000000,43.000000,91.000000,91.000000,91.000000
100067499,23.674874,25.109734,24.272838,118.0,149.0,133.750000,62.0,108.0,85.105691,53.0,...,17.508111,1.279790,1.132793,1.023006,39.439830,44.198080,41.719881,87.175870,139.328809,106.935197
